In [1]:
import pandas as pd
import requests
import numpy as np
from splinter import Browser
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
pd.options.mode.chained_assignment = None  # default='warn'
from time import sleep
from selenium.common.exceptions import NoSuchElementException

In [2]:
df_player_bio = pd.DataFrame(columns=['Player_ID', 'Name', 'rel_link', 'Position'])

In [3]:
years = ['2018', '2019', '2020', '2021']

In [4]:
# Intitialze web driver
from selenium.webdriver.chrome.options import Options
options = Options()
options.binary_location = "C:\Program Files\Google\Chrome Beta\Application\chrome.exe"
driver = webdriver.Chrome(options=options)

In [5]:
def load_page(year):
    driver.get(f'https://fantasydata.com/nfl/fantasy-football-leaders?season={year}&seasontype=1&scope=1&subscope=1&aggregatescope=1&range=3')

In [6]:
def load_total_table():

    # Load Player Ids into table
    element = driver.find_element(By.XPATH,'/html/body/div[6]/div/div[3]/div[2]/div/div[2]/div/div/div[2]/section/div[2]/div[3]/div[2]/div[2]/label[3]/label/span')
    element.click()
    sleep(1)
    
    # Select table length to max of 300
    element = driver.find_element(By.XPATH,'/html/body/div[6]/div/div[3]/div[2]/div/div[2]/div/div/div[2]/section/div[2]/div[3]/div[2]/div[2]/a[3]')
    element.click()
    sleep(1)
    
    # kill google pop up ad
    element = driver.find_element(By.XPATH, '//*[@id="mmt-3c7c8681-860c-4f5a-b02c-3798fabee9a6-sticky-close"]')
    element.click()
    sleep(1)

#     pop_up = driver.find_elements(By.XPATH, '//*[@id="mmt-3c7c8681-860c-4f5a-b02c-3798fabee9a6-sticky-close"]')
#     if len(pop_up) > 0:
#             pop_up[0].click()

    # load all table data
    for i in range(10):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        element = driver.find_element(By.LINK_TEXT, 'LOAD MORE')
        sleep(3)
        element.click()
        sleep(3)

In [7]:
def roster_grabber():
    # turn the page into a soup element
    html = driver.page_source
    page = soup(html, 'html.parser')

    # find the player IDs and names and add to dataframe
    # find player id
    results = page.find_all('td', class_='player-id')
    list_ids = [x.text for x in results]
    df_player_bio['Player_ID'] = list_ids
    # df_player_bio
    # print(list_ids)

    # find player name
    results = page.find_all('td', class_='align-left')
    list_name = [x.a.text for x in results]
    df_player_bio = [df_player_bio.append({"Name": x}) for x in results]
    # print(list_name)


    # find relative player link
    list_link = [x.a.get('href') for x in results]
    df_player_bio['rel_link'] = list_link

    # find position in order to filter
    results = page.find_all('span', attrs={'ng-bind': 'dataItem.Position'})
    list_pos = [x.text for x in results]
    df_player_bio['Position'] = list_pos

In [8]:
def drop_positions(df):
    positions = ['QB', 'RB', 'TE', 'WR']
    df = df[df['Position'].isin(positions)]
    df.sort_values('Name', ignore_index=True, inplace=True)
    
    return df
df_player_bio

,Player_ID,Name,rel_link,Position


create a for loop that goes to each url using https://fantasydata.com/nfl/{first_name}-{last_name}-fantasy/{player_id}
creates a soup element from that page, adds the data to the dataframe, then export moves to the next one

In [9]:
def bio_grabber(df_player_bio):

    url = r'https://fantasydata.com'
    count = 0
    total = len(df_player_bio)

    for index, row in df_player_bio.iterrows():
        # testing loop break
    #     if count >= 5:
    #         break
    #     count += 1

        if count != 0 and count % 50 == 0:
            print("***Break Time...***")
            sleep(10)



        # building search url
        player = row['Name']
        player_url = row['rel_link']
        search_url = url + player_url
        print(f'Searching for {player}')

        driver.get(search_url)
        print('Found...Adding Data...')

        # create soup object
        html = driver.page_source
        page = soup(html, 'html.parser')

        # looking for player data
        results = page.find('div', class_='player-info-dl')
        bio = results.find_all('dd')

        # grabbing data
        height_weight = bio[0].span.text
        dob_age = bio[1].span.text
        experience = bio[2].text
        draft = bio[3].text
        college = bio[4].text

        # adding to dataframe
        df_player_bio.loc[index,'height_weight'] = height_weight
        df_player_bio.loc[index, 'dob_age'] = dob_age
        df_player_bio.loc[index,'experience'] = experience
        df_player_bio.loc[index,'draft'] = draft
        df_player_bio.loc[index,'college'] = college
        print(f'Player Data Added: {player}')
        print('--------------------')
        sleep(2)


In [5]:
list_ids = []
list_name = []
list_link = []
list_pos = []

for year in years:
    
    driver.get(f'https://fantasydata.com/nfl/fantasy-football-leaders?season={year}&seasontype=1&scope=1&subscope=1&aggregatescope=1&range=3')
    print(f'Loaded page for {year}')
    
    
    # Load Player Ids into table
    element = driver.find_element(By.XPATH,'/html/body/div[6]/div/div[3]/div[2]/div/div[2]/div/div/div[2]/section/div[2]/div[3]/div[2]/div[2]/label[3]/label/span')
    element.click()
    sleep(1)
    
    # load entire table
    # Select table length to max of 300
    element = driver.find_element(By.XPATH,'/html/body/div[6]/div/div[3]/div[2]/div/div[2]/div/div/div[2]/section/div[2]/div[3]/div[2]/div[2]/a[3]')
    element.click()
    sleep(1)
    
#     # kill google pop up ad
#     element = driver.find_element(By.CLASS_NAME, 'mmt-sticky-close')
#     element.click()
#     sleep(1)

#     pop_up = driver.find_elements(By.ID, 'mmt-3c7c8681-860c-4f5a-b02c-3798fabee9a6-sticky-close')
#     if len(pop_up) > 0:
#             pop_up[0].click()

    # load all table data
    for i in range(10):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(2)
        element = driver.find_element(By.LINK_TEXT, 'LOAD MORE')
        sleep(1)
        element.click()
        sleep(3)
    print(f'Loaded entire table for {year}')
    
    # grab roster
    # turn the page into a soup element
    html = driver.page_source
    page = soup(html, 'html.parser')

    # find the player IDs and names and add to dataframe
    # find player id
    results = page.find_all('td', class_='player-id')
    [list_ids.append(x.text) for x in results]
    
    # df_player_bio
    # print(list_ids)

    # find player name
    results = page.find_all('td', class_='align-left')
    [list_name.append(x.a.text) for x in results]
    
    # print(list_name)


    # find relative player link
    [list_link.append(x.a.get('href')) for x in results]
    

    # find position in order to filter
    results = page.find_all('span', attrs={'ng-bind': 'dataItem.Position'})
    [list_pos.append(x.text) for x in results]
    
    print(f'Grabbed Roster for {year}')
    

df_player_bio['Player_ID'] = list_ids
df_player_bio["Name"] = list_name
df_player_bio['rel_link'] = list_link
df_player_bio['Position'] = list_pos

    
# drop unnecessary positions
positions = ['QB', 'RB', 'TE', 'WR']
df_player_bio = df_player_bio[df_player_bio['Position'].isin(positions)]
df_player_bio.sort_values('Name', ignore_index=True, inplace=True)
print(f'Dropped unnecessary positions...')

# drop duplicate players
df_player_bio.drop_duplicates(subset=['Player_ID'], inplace=True)

# grab bio data for dataframe
    
url = r'https://fantasydata.com'
count = 0
total = len(df_player_bio)

for index, row in df_player_bio.iterrows():
    # testing loop break
#     if count >= 5:
#         break
#     count += 1

    if count != 0 and count % 50 == 0:
        print("***Break Time...***")
        sleep(10)

    # building search url
    player = row['Name']
    player_url = row['rel_link']
    search_url = url + player_url
    print(f'Searching for {player}')

    driver.get(search_url)
    print('Found...Adding Data...')

    # create soup object
    html = driver.page_source
    page = soup(html, 'html.parser')

    # looking for player data
    results = page.find('div', class_='player-info-dl')
    bio = results.find_all('dd')

    # grabbing data
    height_weight = bio[0].span.text
    dob_age = bio[1].span.text
    experience = bio[2].text
    draft = bio[3].text
    college = bio[4].text

    # adding to dataframe
    df_player_bio.loc[index,'height_weight'] = height_weight
    df_player_bio.loc[index, 'dob_age'] = dob_age
    df_player_bio.loc[index,'experience'] = experience
    df_player_bio.loc[index,'draft'] = draft
    df_player_bio.loc[index,'college'] = college
    print(f'Player Data Added: {player}')
    print('--------------------')
    sleep(2)

# exit the driver
driver.quit()

Loaded page for 2018
Loaded entire table for 2018
Grabbed Roster for 2018
Loaded page for 2019
Loaded entire table for 2019
Grabbed Roster for 2019
Loaded page for 2020
Loaded entire table for 2020
Grabbed Roster for 2020
Loaded page for 2021
Loaded entire table for 2021
Grabbed Roster for 2021
Dropped unnecessary positions...
Searching for A.J. Brown
Found...Adding Data...
Player Data Added: A.J. Brown
--------------------
Searching for A.J. Green
Found...Adding Data...
Player Data Added: A.J. Green
--------------------
Searching for AJ Derby
Found...Adding Data...
Player Data Added: AJ Derby
--------------------
Searching for AJ Dillon
Found...Adding Data...
Player Data Added: AJ Dillon
--------------------
Searching for AJ McCarron
Found...Adding Data...
Player Data Added: AJ McCarron
--------------------
Searching for Aaron Fuller
Found...Adding Data...
Player Data Added: Aaron Fuller
--------------------
Searching for Aaron Jones
Found...Adding Data...
Player Data Added: Aaron Jon

Searching for Ben Skowronek
Found...Adding Data...
Player Data Added: Ben Skowronek
--------------------
Searching for Benjamin Cunningham
Found...Adding Data...
Player Data Added: Benjamin Cunningham
--------------------
Searching for Benjamin Watson
Found...Adding Data...
Player Data Added: Benjamin Watson
--------------------
Searching for Bennie Fowler
Found...Adding Data...
Player Data Added: Bennie Fowler
--------------------
Searching for Benny Snell Jr.
Found...Adding Data...
Player Data Added: Benny Snell Jr.
--------------------
Searching for Bilal Powell
Found...Adding Data...
Player Data Added: Bilal Powell
--------------------
Searching for Bisi Johnson
Found...Adding Data...
Player Data Added: Bisi Johnson
--------------------
Searching for Blaine Gabbert
Found...Adding Data...
Player Data Added: Blaine Gabbert
--------------------
Searching for Blake Bell
Found...Adding Data...
Player Data Added: Blake Bell
--------------------
Searching for Blake Bortles
Found...Adding 

Found...Adding Data...
Player Data Added: Charles Jones
--------------------
Searching for Charlie Woerner
Found...Adding Data...
Player Data Added: Charlie Woerner
--------------------
Searching for Charone Peake
Found...Adding Data...
Player Data Added: Charone Peake
--------------------
Searching for Chase Claypool
Found...Adding Data...
Player Data Added: Chase Claypool
--------------------
Searching for Chase Daniel
Found...Adding Data...
Player Data Added: Chase Daniel
--------------------
Searching for Chase Edmonds
Found...Adding Data...
Player Data Added: Chase Edmonds
--------------------
Searching for Chester Rogers
Found...Adding Data...
Player Data Added: Chester Rogers
--------------------
Searching for Chris Carson
Found...Adding Data...
Player Data Added: Chris Carson
--------------------
Searching for Chris Conley
Found...Adding Data...
Player Data Added: Chris Conley
--------------------
Searching for Chris Evans
Found...Adding Data...
Player Data Added: Chris Evans
-

Searching for Daniel Jones
Found...Adding Data...
Player Data Added: Daniel Jones
--------------------
Searching for Danny Amendola
Found...Adding Data...
Player Data Added: Danny Amendola
--------------------
Searching for Dante Pettis
Found...Adding Data...
Player Data Added: Dante Pettis
--------------------
Searching for Dare Ogunbowale
Found...Adding Data...
Player Data Added: Dare Ogunbowale
--------------------
Searching for Darius Bradwell
Found...Adding Data...
Player Data Added: Darius Bradwell
--------------------
Searching for Darius Jackson
Found...Adding Data...
Player Data Added: Darius Jackson
--------------------
Searching for Darius Jennings
Found...Adding Data...
Player Data Added: Darius Jennings
--------------------
Searching for Darius Slayton
Found...Adding Data...
Player Data Added: Darius Slayton
--------------------
Searching for Darnell Mooney
Found...Adding Data...
Player Data Added: Darnell Mooney
--------------------
Searching for Darrel Williams
Found...A

Searching for Devin Asiasi
Found...Adding Data...
Player Data Added: Devin Asiasi
--------------------
Searching for Devin Duvernay
Found...Adding Data...
Player Data Added: Devin Duvernay
--------------------
Searching for Devin Funchess
Found...Adding Data...
Player Data Added: Devin Funchess
--------------------
Searching for Devin Singletary
Found...Adding Data...
Player Data Added: Devin Singletary
--------------------
Searching for Devin Smith
Found...Adding Data...
Player Data Added: Devin Smith
--------------------
Searching for Devine Ozigbo
Found...Adding Data...
Player Data Added: Devine Ozigbo
--------------------
Searching for Devlin Hodges
Found...Adding Data...
Player Data Added: Devlin Hodges
--------------------
Searching for Devonta Freeman
Found...Adding Data...
Player Data Added: Devonta Freeman
--------------------
Searching for Devontae Booker
Found...Adding Data...
Player Data Added: Devontae Booker
--------------------
Searching for Dexter Williams
Found...Addin

...

Searching for Marquez Callaway
Found...Adding Data...
Player Data Added: Marquez Callaway
--------------------
Searching for Marquez Stevenson
Found...Adding Data...
Player Data Added: Marquez Stevenson
--------------------
Searching for Marquez Valdes-Scantling
Found...Adding Data...
Player Data Added: Marquez Valdes-Scantling
--------------------
Searching for Marquise Brown
Found...Adding Data...
Player Data Added: Marquise Brown
--------------------
Searching for Marquise Goodwin
Found...Adding Data...
Player Data Added: Marquise Goodwin
--------------------
Searching for Marshawn Lynch
Found...Adding Data...
Player Data Added: Marshawn Lynch
--------------------
Searching for Martavis Bryant
Found...Adding Data...
Player Data Added: Martavis Bryant
--------------------
Searching for Marvin Hall
Found...Adding Data...
Player Data Added: Marvin Hall
--------------------
Searching for Marvin Jones Jr.
Found...Adding Data...
Player Data Added: Marvin Jones Jr.
--------------------
Sea

Searching for Nick Keizer
Found...Adding Data...
Player Data Added: Nick Keizer
--------------------
Searching for Nick Mullens
Found...Adding Data...
Player Data Added: Nick Mullens
--------------------
Searching for Nick O'Leary
Found...Adding Data...
Player Data Added: Nick O'Leary
--------------------
Searching for Nick Vannett
Found...Adding Data...
Player Data Added: Nick Vannett
--------------------
Searching for Nick Westbrook-Ikhine
Found...Adding Data...
Player Data Added: Nick Westbrook-Ikhine
--------------------
Searching for Nick Williams
Found...Adding Data...
Player Data Added: Nick Williams
--------------------
Searching for Nico Collins
Found...Adding Data...
Player Data Added: Nico Collins
--------------------
Searching for Niles Paul
Found...Adding Data...
Player Data Added: Niles Paul
--------------------
Searching for Noah Brown
Found...Adding Data...
Player Data Added: Noah Brown
--------------------
Searching for Noah Fant
Found...Adding Data...
Player Data Adde

Searching for Rodney Adams
Found...Adding Data...
Player Data Added: Rodney Adams
--------------------
Searching for Rodney Smith
Found...Adding Data...
Player Data Added: Rodney Smith
--------------------
Searching for Ronald Jones
Found...Adding Data...
Player Data Added: Ronald Jones
--------------------
Searching for Rondale Moore
Found...Adding Data...
Player Data Added: Rondale Moore
--------------------
Searching for Ross Dwelley
Found...Adding Data...
Player Data Added: Ross Dwelley
--------------------
Searching for Ross Travis
Found...Adding Data...
Player Data Added: Ross Travis
--------------------
Searching for Royce Freeman
Found...Adding Data...
Player Data Added: Royce Freeman
--------------------
Searching for Russell Gage
Found...Adding Data...
Player Data Added: Russell Gage
--------------------
Searching for Russell Shepard
Found...Adding Data...
Player Data Added: Russell Shepard
--------------------
Searching for Russell Wilson
Found...Adding Data...
Player Data A

Searching for Teddy Bridgewater
Found...Adding Data...
Player Data Added: Teddy Bridgewater
--------------------
Searching for Tee Higgins
Found...Adding Data...
Player Data Added: Tee Higgins
--------------------
Searching for Temarrick Hemingway
Found...Adding Data...
Player Data Added: Temarrick Hemingway
--------------------
Searching for Terrace Marshall Jr.
Found...Adding Data...
Player Data Added: Terrace Marshall Jr.
--------------------
Searching for Terrance Williams
Found...Adding Data...
Player Data Added: Terrance Williams
--------------------
Searching for Terrelle Pryor
Found...Adding Data...
Player Data Added: Terrelle Pryor
--------------------
Searching for Terry Godwin
Found...Adding Data...
Player Data Added: Terry Godwin
--------------------
Searching for Terry McLaurin
Found...Adding Data...
Player Data Added: Terry McLaurin
--------------------
Searching for Tevin Coleman
Found...Adding Data...
Player Data Added: Tevin Coleman
--------------------
Searching for T

Searching for Tyrie Cleveland
Found...Adding Data...
Player Data Added: Tyrie Cleveland
--------------------
Searching for Tyrod Taylor
Found...Adding Data...
Player Data Added: Tyrod Taylor
--------------------
Searching for Tyron Johnson
Found...Adding Data...
Player Data Added: Tyron Johnson
--------------------
Searching for Tyrone Swoopes
Found...Adding Data...
Player Data Added: Tyrone Swoopes
--------------------
Searching for Van Jefferson
Found...Adding Data...
Player Data Added: Van Jefferson
--------------------
Searching for Vance McDonald
Found...Adding Data...
Player Data Added: Vance McDonald
--------------------
Searching for Ventell Bryant
Found...Adding Data...
Player Data Added: Ventell Bryant
--------------------
Searching for Vernon Davis
Found...Adding Data...
Player Data Added: Vernon Davis
--------------------
Searching for Victor Bolden
Found...Adding Data...
Player Data Added: Victor Bolden
--------------------
Searching for Virgil Green
Found...Adding Data...

In [6]:
# display dataframe
df_player_bio

,Player_ID,Name,rel_link,Position,height_weight,dob_age,experience,draft,college
0,21042,A.J. Brown,/nfl/aj-brown-fantasy/21042,WR,"6'0"" / 226 lbs.","June 30, 1997 (25)",4th Season,2019 / Rd. 2 / Pk. 51 (TEN),Mississippi
3,12845,A.J. Green,/nfl/aj-green-fantasy/12845,WR,"6'4"" / 207 lbs.","July 31, 1988 (33)",12th Season,2011 / Rd. 1 / Pk. 4 (CIN),Georgia
6,16962,AJ Derby,/nfl/aj-derby-fantasy/16962,TE,"6'5"" / 245 lbs.","September 20, 1991 (30)",5th Season,2015 / Rd. 6 / Pk. 202 (NE),Arkansas
7,21802,AJ Dillon,/nfl/aj-dillon-fantasy/21802,RB,"6'0"" / 247 lbs.","May 2, 1998 (24)",3rd Season,2020 / Rd. 2 / Pk. 62 (GB),Boston College
9,16116,AJ McCarron,/nfl/aj-mccarron-fantasy/16116,QB,"6'3"" / 215 lbs.","September 13, 1990 (31)",8th Season,2014 / Rd. 5 / Pk. 164 (CIN),Alabama
...,...,...,...,...,...,...,...,...,...
2467,19172,Zach Pascal,/nfl/zach-pascal-fantasy/19172,WR,"6'2"" / 214 lbs.","December 18, 1994 (27)",5th Season,2017 / Rd. / Pk. (WAS),Old Dominion
2471,22495,Zach Wilson,/nfl/zach-wilson-fantasy/22495,QB,"6'2"" / 214 lbs.","August 3, 1999 (22)",2nd Season,2021 / Rd. 1 / Pk. 2 (NYJ),Brigham Young
2472,17023,Zach Zenner,/nfl/zach-zenner-fantasy/17023,RB,"5'11"" / 223 lbs.","September 13, 1991 (30)",5th Season,2015 / Rd. / Pk. (DET),South Dakota State
2474,21784,Zack Moss,/nfl/zack-moss-fantasy/21784,RB,"5'9"" / 205 lbs.","December 15, 1997 (24)",3rd Season,2020 / Rd. 3 / Pk. 86 (BUF),Utah


In [14]:
# export to csv to save scrape
file_save = '../Resources/player_bio_data.csv'
df_player_bio.to_csv(file_save, index=False, na_rep='NULL')

In [9]:
import re


for index, row in df_player_bio.iterrows():
    str_list = row['height_weight'].split()
    height = str_list[0]
    weight = str_list[2]
    weight = re.sub('lbs.*', '', weight)
    
    df_player_bio.loc[index,'height'] = height
    df_player_bio.loc[index, 'weight_lbs'] = weight
    

df_player_bio

,Player_ID,Name,rel_link,Position,height_weight,dob_age,experience,draft,college,height,weight_lbs
0,21042,A.J. Brown,/nfl/aj-brown-fantasy/21042,WR,"6'0"" / 226 lbs.","June 30, 1997 (25)",4th Season,2019 / Rd. 2 / Pk. 51 (TEN),Mississippi,"6'0""",226
3,12845,A.J. Green,/nfl/aj-green-fantasy/12845,WR,"6'4"" / 207 lbs.","July 31, 1988 (33)",12th Season,2011 / Rd. 1 / Pk. 4 (CIN),Georgia,"6'4""",207
6,16962,AJ Derby,/nfl/aj-derby-fantasy/16962,TE,"6'5"" / 245 lbs.","September 20, 1991 (30)",5th Season,2015 / Rd. 6 / Pk. 202 (NE),Arkansas,"6'5""",245
7,21802,AJ Dillon,/nfl/aj-dillon-fantasy/21802,RB,"6'0"" / 247 lbs.","May 2, 1998 (24)",3rd Season,2020 / Rd. 2 / Pk. 62 (GB),Boston College,"6'0""",247
9,16116,AJ McCarron,/nfl/aj-mccarron-fantasy/16116,QB,"6'3"" / 215 lbs.","September 13, 1990 (31)",8th Season,2014 / Rd. 5 / Pk. 164 (CIN),Alabama,"6'3""",215
...,...,...,...,...,...,...,...,...,...,...,...
2467,19172,Zach Pascal,/nfl/zach-pascal-fantasy/19172,WR,"6'2"" / 214 lbs.","December 18, 1994 (27)",5th Season,2017 / Rd. / Pk. (WAS),Old Dominion,"6'2""",214
2471,22495,Zach Wilson,/nfl/zach-wilson-fantasy/22495,QB,"6'2"" / 214 lbs.","August 3, 1999 (22)",2nd Season,2021 / Rd. 1 / Pk. 2 (NYJ),Brigham Young,"6'2""",214
2472,17023,Zach Zenner,/nfl/zach-zenner-fantasy/17023,RB,"5'11"" / 223 lbs.","September 13, 1991 (30)",5th Season,2015 / Rd. / Pk. (DET),South Dakota State,"5'11""",223
2474,21784,Zack Moss,/nfl/zack-moss-fantasy/21784,RB,"5'9"" / 205 lbs.","December 15, 1997 (24)",3rd Season,2020 / Rd. 3 / Pk. 86 (BUF),Utah,"5'9""",205


In [13]:
for index, row in df_player_bio.iterrows():
    str_list = row['dob_age'].rsplit(maxsplit=1)
    dob = str_list[0]
    df_player_bio.loc[index, 'dob'] = dob
    
df_player_bio
    

,Player_ID,Name,rel_link,Position,height_weight,dob_age,experience,draft,college,height,weight_lbs,dob
0,21042,A.J. Brown,/nfl/aj-brown-fantasy/21042,WR,"6'0"" / 226 lbs.","June 30, 1997 (25)",4th Season,2019 / Rd. 2 / Pk. 51 (TEN),Mississippi,"6'0""",226,"June 30, 1997"
3,12845,A.J. Green,/nfl/aj-green-fantasy/12845,WR,"6'4"" / 207 lbs.","July 31, 1988 (33)",12th Season,2011 / Rd. 1 / Pk. 4 (CIN),Georgia,"6'4""",207,"July 31, 1988"
6,16962,AJ Derby,/nfl/aj-derby-fantasy/16962,TE,"6'5"" / 245 lbs.","September 20, 1991 (30)",5th Season,2015 / Rd. 6 / Pk. 202 (NE),Arkansas,"6'5""",245,"September 20, 1991"
7,21802,AJ Dillon,/nfl/aj-dillon-fantasy/21802,RB,"6'0"" / 247 lbs.","May 2, 1998 (24)",3rd Season,2020 / Rd. 2 / Pk. 62 (GB),Boston College,"6'0""",247,"May 2, 1998"
9,16116,AJ McCarron,/nfl/aj-mccarron-fantasy/16116,QB,"6'3"" / 215 lbs.","September 13, 1990 (31)",8th Season,2014 / Rd. 5 / Pk. 164 (CIN),Alabama,"6'3""",215,"September 13, 1990"
...,...,...,...,...,...,...,...,...,...,...,...,...
2467,19172,Zach Pascal,/nfl/zach-pascal-fantasy/19172,WR,"6'2"" / 214 lbs.","December 18, 1994 (27)",5th Season,2017 / Rd. / Pk. (WAS),Old Dominion,"6'2""",214,"December 18, 1994"
2471,22495,Zach Wilson,/nfl/zach-wilson-fantasy/22495,QB,"6'2"" / 214 lbs.","August 3, 1999 (22)",2nd Season,2021 / Rd. 1 / Pk. 2 (NYJ),Brigham Young,"6'2""",214,"August 3, 1999"
2472,17023,Zach Zenner,/nfl/zach-zenner-fantasy/17023,RB,"5'11"" / 223 lbs.","September 13, 1991 (30)",5th Season,2015 / Rd. / Pk. (DET),South Dakota State,"5'11""",223,"September 13, 1991"
2474,21784,Zack Moss,/nfl/zack-moss-fantasy/21784,RB,"5'9"" / 205 lbs.","December 15, 1997 (24)",3rd Season,2020 / Rd. 3 / Pk. 86 (BUF),Utah,"5'9""",205,"December 15, 1997"


In [14]:
df_player_bio['dob'] = pd.to_datetime(df_player_bio['dob'])

In [15]:
df_player_bio['dob'] = df_player_bio['dob'].dt.strftime('%m/%d/%Y')
df_player_bio

,Player_ID,Name,rel_link,Position,height_weight,dob_age,experience,draft,college,height,weight_lbs,dob
0,21042,A.J. Brown,/nfl/aj-brown-fantasy/21042,WR,"6'0"" / 226 lbs.","June 30, 1997 (25)",4th Season,2019 / Rd. 2 / Pk. 51 (TEN),Mississippi,"6'0""",226,06/30/1997
3,12845,A.J. Green,/nfl/aj-green-fantasy/12845,WR,"6'4"" / 207 lbs.","July 31, 1988 (33)",12th Season,2011 / Rd. 1 / Pk. 4 (CIN),Georgia,"6'4""",207,07/31/1988
6,16962,AJ Derby,/nfl/aj-derby-fantasy/16962,TE,"6'5"" / 245 lbs.","September 20, 1991 (30)",5th Season,2015 / Rd. 6 / Pk. 202 (NE),Arkansas,"6'5""",245,09/20/1991
7,21802,AJ Dillon,/nfl/aj-dillon-fantasy/21802,RB,"6'0"" / 247 lbs.","May 2, 1998 (24)",3rd Season,2020 / Rd. 2 / Pk. 62 (GB),Boston College,"6'0""",247,05/02/1998
9,16116,AJ McCarron,/nfl/aj-mccarron-fantasy/16116,QB,"6'3"" / 215 lbs.","September 13, 1990 (31)",8th Season,2014 / Rd. 5 / Pk. 164 (CIN),Alabama,"6'3""",215,09/13/1990
...,...,...,...,...,...,...,...,...,...,...,...,...
2467,19172,Zach Pascal,/nfl/zach-pascal-fantasy/19172,WR,"6'2"" / 214 lbs.","December 18, 1994 (27)",5th Season,2017 / Rd. / Pk. (WAS),Old Dominion,"6'2""",214,12/18/1994
2471,22495,Zach Wilson,/nfl/zach-wilson-fantasy/22495,QB,"6'2"" / 214 lbs.","August 3, 1999 (22)",2nd Season,2021 / Rd. 1 / Pk. 2 (NYJ),Brigham Young,"6'2""",214,08/03/1999
2472,17023,Zach Zenner,/nfl/zach-zenner-fantasy/17023,RB,"5'11"" / 223 lbs.","September 13, 1991 (30)",5th Season,2015 / Rd. / Pk. (DET),South Dakota State,"5'11""",223,09/13/1991
2474,21784,Zack Moss,/nfl/zack-moss-fantasy/21784,RB,"5'9"" / 205 lbs.","December 15, 1997 (24)",3rd Season,2020 / Rd. 3 / Pk. 86 (BUF),Utah,"5'9""",205,12/15/1997


In [16]:
for index, row in df_player_bio.iterrows():
    exp = re.sub('\D*', '', row['experience'])
    df_player_bio.loc[index, 'exp(y)'] = exp
    
df_player_bio

,Player_ID,Name,rel_link,Position,height_weight,dob_age,experience,draft,college,height,weight_lbs,dob,exp(y)
0,21042,A.J. Brown,/nfl/aj-brown-fantasy/21042,WR,"6'0"" / 226 lbs.","June 30, 1997 (25)",4th Season,2019 / Rd. 2 / Pk. 51 (TEN),Mississippi,"6'0""",226,06/30/1997,4
3,12845,A.J. Green,/nfl/aj-green-fantasy/12845,WR,"6'4"" / 207 lbs.","July 31, 1988 (33)",12th Season,2011 / Rd. 1 / Pk. 4 (CIN),Georgia,"6'4""",207,07/31/1988,12
6,16962,AJ Derby,/nfl/aj-derby-fantasy/16962,TE,"6'5"" / 245 lbs.","September 20, 1991 (30)",5th Season,2015 / Rd. 6 / Pk. 202 (NE),Arkansas,"6'5""",245,09/20/1991,5
7,21802,AJ Dillon,/nfl/aj-dillon-fantasy/21802,RB,"6'0"" / 247 lbs.","May 2, 1998 (24)",3rd Season,2020 / Rd. 2 / Pk. 62 (GB),Boston College,"6'0""",247,05/02/1998,3
9,16116,AJ McCarron,/nfl/aj-mccarron-fantasy/16116,QB,"6'3"" / 215 lbs.","September 13, 1990 (31)",8th Season,2014 / Rd. 5 / Pk. 164 (CIN),Alabama,"6'3""",215,09/13/1990,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2467,19172,Zach Pascal,/nfl/zach-pascal-fantasy/19172,WR,"6'2"" / 214 lbs.","December 18, 1994 (27)",5th Season,2017 / Rd. / Pk. (WAS),Old Dominion,"6'2""",214,12/18/1994,5
2471,22495,Zach Wilson,/nfl/zach-wilson-fantasy/22495,QB,"6'2"" / 214 lbs.","August 3, 1999 (22)",2nd Season,2021 / Rd. 1 / Pk. 2 (NYJ),Brigham Young,"6'2""",214,08/03/1999,2
2472,17023,Zach Zenner,/nfl/zach-zenner-fantasy/17023,RB,"5'11"" / 223 lbs.","September 13, 1991 (30)",5th Season,2015 / Rd. / Pk. (DET),South Dakota State,"5'11""",223,09/13/1991,5
2474,21784,Zack Moss,/nfl/zack-moss-fantasy/21784,RB,"5'9"" / 205 lbs.","December 15, 1997 (24)",3rd Season,2020 / Rd. 3 / Pk. 86 (BUF),Utah,"5'9""",205,12/15/1997,3


In [17]:
# export to csv to save regex
file_save = '../Resources/player_bio_data.csv'
df_player_bio.to_csv(file_save, index=False, na_rep='NULL')

In [31]:
df_player_bio = pd.read_csv(file_save)
df_player_bio

,Player_ID,Name,rel_link,Position,height_weight,dob_age,experience,draft,college,height,weight_lbs,dob,exp(y)
0,21042,A.J. Brown,/nfl/aj-brown-fantasy/21042,WR,"6'0"" / 226 lbs.","June 30, 1997 (25)",4th Season,2019 / Rd. 2 / Pk. 51 (TEN),Mississippi,"6'0""",226,06/30/1997,4
1,12845,A.J. Green,/nfl/aj-green-fantasy/12845,WR,"6'4"" / 207 lbs.","July 31, 1988 (33)",12th Season,2011 / Rd. 1 / Pk. 4 (CIN),Georgia,"6'4""",207,07/31/1988,12
2,16962,AJ Derby,/nfl/aj-derby-fantasy/16962,TE,"6'5"" / 245 lbs.","September 20, 1991 (30)",5th Season,2015 / Rd. 6 / Pk. 202 (NE),Arkansas,"6'5""",245,09/20/1991,5
3,21802,AJ Dillon,/nfl/aj-dillon-fantasy/21802,RB,"6'0"" / 247 lbs.","May 2, 1998 (24)",3rd Season,2020 / Rd. 2 / Pk. 62 (GB),Boston College,"6'0""",247,05/02/1998,3
4,16116,AJ McCarron,/nfl/aj-mccarron-fantasy/16116,QB,"6'3"" / 215 lbs.","September 13, 1990 (31)",8th Season,2014 / Rd. 5 / Pk. 164 (CIN),Alabama,"6'3""",215,09/13/1990,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038,19172,Zach Pascal,/nfl/zach-pascal-fantasy/19172,WR,"6'2"" / 214 lbs.","December 18, 1994 (27)",5th Season,2017 / Rd. / Pk. (WAS),Old Dominion,"6'2""",214,12/18/1994,5
1039,22495,Zach Wilson,/nfl/zach-wilson-fantasy/22495,QB,"6'2"" / 214 lbs.","August 3, 1999 (22)",2nd Season,2021 / Rd. 1 / Pk. 2 (NYJ),Brigham Young,"6'2""",214,08/03/1999,2
1040,17023,Zach Zenner,/nfl/zach-zenner-fantasy/17023,RB,"5'11"" / 223 lbs.","September 13, 1991 (30)",5th Season,2015 / Rd. / Pk. (DET),South Dakota State,"5'11""",223,09/13/1991,5
1041,21784,Zack Moss,/nfl/zack-moss-fantasy/21784,RB,"5'9"" / 205 lbs.","December 15, 1997 (24)",3rd Season,2020 / Rd. 3 / Pk. 86 (BUF),Utah,"5'9""",205,12/15/1997,3


In [19]:
drop_columns = ['height_weight', 'dob_age', 'experience']
df_player_bio.drop(columns=drop_columns, inplace=True)
df_player_bio

,Player_ID,Name,rel_link,Position,draft,college,height,weight_lbs,dob,exp(y)
0,21042,A.J. Brown,/nfl/aj-brown-fantasy/21042,WR,2019 / Rd. 2 / Pk. 51 (TEN),Mississippi,"6'0""",226,06/30/1997,4
3,12845,A.J. Green,/nfl/aj-green-fantasy/12845,WR,2011 / Rd. 1 / Pk. 4 (CIN),Georgia,"6'4""",207,07/31/1988,12
6,16962,AJ Derby,/nfl/aj-derby-fantasy/16962,TE,2015 / Rd. 6 / Pk. 202 (NE),Arkansas,"6'5""",245,09/20/1991,5
7,21802,AJ Dillon,/nfl/aj-dillon-fantasy/21802,RB,2020 / Rd. 2 / Pk. 62 (GB),Boston College,"6'0""",247,05/02/1998,3
9,16116,AJ McCarron,/nfl/aj-mccarron-fantasy/16116,QB,2014 / Rd. 5 / Pk. 164 (CIN),Alabama,"6'3""",215,09/13/1990,8
...,...,...,...,...,...,...,...,...,...,...
2467,19172,Zach Pascal,/nfl/zach-pascal-fantasy/19172,WR,2017 / Rd. / Pk. (WAS),Old Dominion,"6'2""",214,12/18/1994,5
2471,22495,Zach Wilson,/nfl/zach-wilson-fantasy/22495,QB,2021 / Rd. 1 / Pk. 2 (NYJ),Brigham Young,"6'2""",214,08/03/1999,2
2472,17023,Zach Zenner,/nfl/zach-zenner-fantasy/17023,RB,2015 / Rd. / Pk. (DET),South Dakota State,"5'11""",223,09/13/1991,5
2474,21784,Zack Moss,/nfl/zack-moss-fantasy/21784,RB,2020 / Rd. 3 / Pk. 86 (BUF),Utah,"5'9""",205,12/15/1997,3


In [36]:
for index, row in df_player_bio.iterrows():
    str_list = row['draft'].split()
    
    if len(str_list) == 8:
        draft_year = str_list[0]
        draft_round = str_list[3]
        draft_pick = str_list[6]
        draft_team = str_list[7]

        draft_team = draft_team.lstrip('(').rstrip(')')

        df_player_bio.loc[index, 'draft_year'] = draft_year
        df_player_bio.loc[index, 'draft_round'] = draft_round
        df_player_bio.loc[index, 'draft_pick'] = draft_pick
        df_player_bio.loc[index, 'draft_team'] = draft_team
    else: 
        draft_year = str_list[0]
        draft_team = str_list[5]
        
        draft_team = draft_team.lstrip('(').rstrip(')')
        
        df_player_bio.loc[index, 'draft_year'] = draft_year
        df_player_bio.loc[index, 'draft_round'] = np.nan
        df_player_bio.loc[index, 'draft_pick'] = np.nan
        df_player_bio.loc[index, 'draft_team'] = draft_team
df_player_bio

,Player_ID,Name,rel_link,Position,height_weight,dob_age,experience,draft,college,height,weight_lbs,dob,exp(y),draft_year,draft_round,draft_pick,draft_team
0,21042,A.J. Brown,/nfl/aj-brown-fantasy/21042,WR,"6'0"" / 226 lbs.","June 30, 1997 (25)",4th Season,2019 / Rd. 2 / Pk. 51 (TEN),Mississippi,"6'0""",226,06/30/1997,4,2019,2,51,TEN
1,12845,A.J. Green,/nfl/aj-green-fantasy/12845,WR,"6'4"" / 207 lbs.","July 31, 1988 (33)",12th Season,2011 / Rd. 1 / Pk. 4 (CIN),Georgia,"6'4""",207,07/31/1988,12,2011,1,4,CIN
2,16962,AJ Derby,/nfl/aj-derby-fantasy/16962,TE,"6'5"" / 245 lbs.","September 20, 1991 (30)",5th Season,2015 / Rd. 6 / Pk. 202 (NE),Arkansas,"6'5""",245,09/20/1991,5,2015,6,202,NE
3,21802,AJ Dillon,/nfl/aj-dillon-fantasy/21802,RB,"6'0"" / 247 lbs.","May 2, 1998 (24)",3rd Season,2020 / Rd. 2 / Pk. 62 (GB),Boston College,"6'0""",247,05/02/1998,3,2020,2,62,GB
4,16116,AJ McCarron,/nfl/aj-mccarron-fantasy/16116,QB,"6'3"" / 215 lbs.","September 13, 1990 (31)",8th Season,2014 / Rd. 5 / Pk. 164 (CIN),Alabama,"6'3""",215,09/13/1990,8,2014,5,164,CIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038,19172,Zach Pascal,/nfl/zach-pascal-fantasy/19172,WR,"6'2"" / 214 lbs.","December 18, 1994 (27)",5th Season,2017 / Rd. / Pk. (WAS),Old Dominion,"6'2""",214,12/18/1994,5,2017,NaN,NaN,WAS
1039,22495,Zach Wilson,/nfl/zach-wilson-fantasy/22495,QB,"6'2"" / 214 lbs.","August 3, 1999 (22)",2nd Season,2021 / Rd. 1 / Pk. 2 (NYJ),Brigham Young,"6'2""",214,08/03/1999,2,2021,1,2,NYJ
1040,17023,Zach Zenner,/nfl/zach-zenner-fantasy/17023,RB,"5'11"" / 223 lbs.","September 13, 1991 (30)",5th Season,2015 / Rd. / Pk. (DET),South Dakota State,"5'11""",223,09/13/1991,5,2015,NaN,NaN,DET
1041,21784,Zack Moss,/nfl/zack-moss-fantasy/21784,RB,"5'9"" / 205 lbs.","December 15, 1997 (24)",3rd Season,2020 / Rd. 3 / Pk. 86 (BUF),Utah,"5'9""",205,12/15/1997,3,2020,3,86,BUF


In [37]:
df_player_bio.drop(columns='draft', inplace=True)
df_player_bio

,Player_ID,Name,rel_link,Position,height_weight,dob_age,experience,college,height,weight_lbs,dob,exp(y),draft_year,draft_round,draft_pick,draft_team
0,21042,A.J. Brown,/nfl/aj-brown-fantasy/21042,WR,"6'0"" / 226 lbs.","June 30, 1997 (25)",4th Season,Mississippi,"6'0""",226,06/30/1997,4,2019,2,51,TEN
1,12845,A.J. Green,/nfl/aj-green-fantasy/12845,WR,"6'4"" / 207 lbs.","July 31, 1988 (33)",12th Season,Georgia,"6'4""",207,07/31/1988,12,2011,1,4,CIN
2,16962,AJ Derby,/nfl/aj-derby-fantasy/16962,TE,"6'5"" / 245 lbs.","September 20, 1991 (30)",5th Season,Arkansas,"6'5""",245,09/20/1991,5,2015,6,202,NE
3,21802,AJ Dillon,/nfl/aj-dillon-fantasy/21802,RB,"6'0"" / 247 lbs.","May 2, 1998 (24)",3rd Season,Boston College,"6'0""",247,05/02/1998,3,2020,2,62,GB
4,16116,AJ McCarron,/nfl/aj-mccarron-fantasy/16116,QB,"6'3"" / 215 lbs.","September 13, 1990 (31)",8th Season,Alabama,"6'3""",215,09/13/1990,8,2014,5,164,CIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038,19172,Zach Pascal,/nfl/zach-pascal-fantasy/19172,WR,"6'2"" / 214 lbs.","December 18, 1994 (27)",5th Season,Old Dominion,"6'2""",214,12/18/1994,5,2017,NaN,NaN,WAS
1039,22495,Zach Wilson,/nfl/zach-wilson-fantasy/22495,QB,"6'2"" / 214 lbs.","August 3, 1999 (22)",2nd Season,Brigham Young,"6'2""",214,08/03/1999,2,2021,1,2,NYJ
1040,17023,Zach Zenner,/nfl/zach-zenner-fantasy/17023,RB,"5'11"" / 223 lbs.","September 13, 1991 (30)",5th Season,South Dakota State,"5'11""",223,09/13/1991,5,2015,NaN,NaN,DET
1041,21784,Zack Moss,/nfl/zack-moss-fantasy/21784,RB,"5'9"" / 205 lbs.","December 15, 1997 (24)",3rd Season,Utah,"5'9""",205,12/15/1997,3,2020,3,86,BUF


In [39]:
# export to csv to save scrape
file_save = '../Resources/player_bio_data.csv'
df_player_bio.to_csv(file_save, index=False, na_rep='NULL')